In [ ]:
import sys
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from copy import deepcopy

In [ ]:
giant_table_path = r"C:\Users\orson\Desktop\Myself\HCI\UWiSchool\Projects\UWEXP\Data\UWEXPI_SampleData\glue"
path_to_save_figs = r"C:\Users\orson\Desktop\Myself\HCI\UWiSchool\Projects\UWEXP\Data\UWEXPI_SampleData\Visualization"

# Data Input and Cleaning

In [ ]:
giant_table = pd.read_csv(giant_table_path + "\giant_table_sensor_sleep_EMA_baseline.csv")
giant_table["date"] = pd.to_datetime(giant_table["date"])
giant_table.set_index("date", inplace = True)
min_date = giant_table.index.min()
max_date = giant_table.index.max()

In [ ]:
'''Chose numeric features to visualize'''
all_features_can_be_visualize = giant_table.columns.tolist()[1:]
all_features_can_be_visualize = [x for x in all_features_can_be_visualize if ((giant_table[x].dtype == np.float64
                                                               or giant_table[x].dtype == np.int64))]

In [ ]:
'''Chose whether filtering is needed or not'''
filtering_falg = False
if (filtering_falg):
    filtering_th = {k:{"down":-1000000, "up":1440000} for k in all_features_can_be_visualize}
    def filtering(data, min_value = 0, max_value = 1000):
        data = data.mask(data < min_value)
        data = data.mask(data > max_value)
        return data
    for feature_to_visualize in all_features_can_be_visualize:
        data = giant_table[[feature_to_visualize]]
        giant_table[feature_to_visualize] = filtering(data, filtering_th[feature_to_visualize]["down"], filtering_th[feature_to_visualize]["up"])

In [ ]:
'''Prepare necessary format for visulization function'''

giant_table_groups = giant_table.groupby("PID")
pids = list(giant_table_groups.groups.keys())

all_pids = list(set(giant_table["PID"]))
dis_pids = list(set(giant_table[giant_table.discriminated == "YES"]["PID"]))
nodis_pids = list(set(all_pids) - set(dis_pids))
flag_pids = {}
for pid in all_pids:
    flag_pids[pid] = False
for pid in dis_pids:
    flag_pids[pid] = True
    
dis_events = deepcopy(giant_table[giant_table.discriminated == "YES"])
dis_events.reset_index(inplace = True)
dis_events.rename({"index":"date"}, axis = 1, inplace = True)
dates_discrimination_pids = {}
for idx, row in dis_events[["PID","date"]].iterrows():
    pid = row["PID"]
    date = row["date"]
    if (pid not in dates_discrimination_pids):
        dates_discrimination_pids[pid] = [date]
    else:
        dates_discrimination_pids[pid].append(date)

# Visualize participants along the timeline

In [ ]:
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)
date_range = pd.date_range(start_date, end_date)
df_date_range = pd.DataFrame(index = date_range)

In [ ]:
def plot_all_participants_together_per_feature(
    table_groups,
    features_to_visualize,
    pids_to_visualize,
    flags_discrimination,
    dates_discrimination,
    start_date=min_date,
    end_date=max_date,
    save_fig_flag = False,
    save_fig_path = path_to_save_figs + "/all_participants_together/",
):
    
    '''
    Visualize all participants' together, one figure per feature
    
    :param DataFrame.groupby table_groups: big tables groupby on pids
    :param list features_to_visualize: a list of feature names (column names) to figures
    :param list pids_to_visualize: a list of pids to be visualized
    :param dict flags_discrimination: a dictionary including all pids about the flag, {key: pids, value: flags}
    :param dict dates_discrimination: a dictionary including all pids about the dates they experience about discrimination
                                            {key: pids who experienced discrimination, value: a list of dates}
    :param str start_date: start date for visualization
    :param str end_date: end date for visualization
    :param bool save_fig_flag: true for save the figure, false to plot
    :param str save_fig_path: path to save figures, if plotted
    :return: a list of figures
    '''
    figs = []
    colors1 = ["#CD2990", "#D02090", "#C71585", "#EE30A7", "#FF34B3", "#FF6EC7", "#F6A4D5", "#F7B3DA", "#543948", "#D4318C", "#8B0A50", "#CD1076", "#EE1289", "#FF1493", "#9D6B84", "#8B3A62", "#872657", "#DE85B1", "#551033", "#FF007F", "#FF69B4", "#FF6EB4", "#A46582", "#EE6AA7", "#B6316C", "#8B2252", "#CD3278", "#EE3A8C", "#FF3E96", "#CD6090", "#862A51", "#FF0066", "#8B8386", "#E0427F", "#FF92BB", "#A2627A", "#B03060", "#EEE0E5", "#CDC1C5", "#CD6889", "#DB7093", "#F6CCDA", "#FF82AB", "#FFF0F5", "#8B475D", "#EE799F", "#EAB5C5", "#A5435C", "#8B636C", "#CD919E", "#EEA9B8", "#FFB5C5", "#DC143C", "#F54D70", "#FF0033", "#F6A8B6", "#DB2645", "#FFC0CB", "#E79EA9", "#EEA2AD", "#99182C", "#E31230", "#FFADB9", "#FFB6C1", "#8B5F65", "#422C2F", "#CD8C95", "#DB9EA6", "#FFAEB9", "#CC4E5C", "#55141C", "#BB2A3C", "#C82536", "#AF1E2D", "#B81324", "#E32636", "#FA1D2F", "#F6C9CC", "#B0171F", "#EB5E66", "#D0A9AA", "#D41A1F", "#F64D54", "#FF030D", "#551011", "#E32E30", "#E33638", "#9E0508", "#E35152"]
    colors2 = ["#008B8B", "#00CDCD", "#00EEEE", "#00FFFF", "#00FFFF", "#97FFFF", "#BBFFFF", "#E0FFFF", "#F0FFFF", "#00CED1", "#5F9EA0", "#00868B", "#00C5CD", "#00E5EE", "#00F5FF", "#67E6EC", "#4A777A", "#05EDFF", "#53868B", "#73B1B7", "#05E9FF", "#7AC5CD", "#8EE5EE", "#05B8CC", "#98F5FF", "#B0E0E6", "#C1F0F6", "#39B7CD", "#65909A", "#0EBFE9", "#C3E4ED", "#68838B", "#63D1F4", "#9AC0CD", "#50A6C2", "#ADD8E6", "#B2DFEE", "#00688B", "#009ACD", "#0099CC", "#00B2EE", "#00BFFF", "#BFEFFF", "#33A1C9", "#507786", "#87CEEB", "#38B0DE", "#0BB5FF", "#42C0FB", "#6996AD", "#539DC2", "#236B8E", "#3299CC", "#0198E1", "#33A1DE", "#607B8B", "#35586C", "#5D92B1", "#8DB6CD", "#325C74", "#A4D3EE", "#82CFFD", "#67C8FF", "#B0E2FF", "#87CEFA", "#6CA6CD", "#4A708B", "#9BC4E2", "#7EC0EE", "#87CEFF", "#517693", "#5D7B93", "#42647F", "#4682B4", "#4F94CD", "#5CACEE", "#63B8FF", "#525C65", "#36648B", "#62B1F6", "#74BBFB", "#F0F8FF", "#4E78A0", "#0D4F8B", "#708090", "#708090", "#778899", "#778899", "#6183A6", "#9FB6CD", "#7D9EC0", "#104E8B", "#1874CD", "#1C86EE", "#60AFFE", "#007FFF", "#1E90FF", "#6C7B8B", "#B7C3D0", "#739AC5", "#75A1D0", "#B9D3EE", "#499DF5", "#C6E2FF", "#3B6AA0", "#7AA9DD", "#0276FD", "#003F87", "#6E7B8B", "#506987", "#A2B5CD", "#4372AA", "#26466D", "#1D7CF2", "#687C97", "#344152", "#50729F", "#4973AB", "#B0C4DE", "#3063A5", "#BCD2EE", "#7EB6FF", "#CAE1FF", "#4D71A3", "#2B4F81", "#4981CE", "#88ACE0", "#5993E5", "#3A66A7", "#3579DC", "#5190ED", "#42526C", "#4D6FAC", "#2E5090", "#2C5197", "#6495ED", "#6D9BF1", "#5B90F6", "#1464F4", "#3A5894", "#7093DB", "#1B3F8B", "#5971AD", "#0147FA", "#3D59AB", "#27408B", "#3A5FCD", "#4169E1", "#436EEE", "#003EFF", "#4876FF", "#A9ACB6", "#22316C", "#162252", "#3B4990", "#283A90", "#6F7285", "#838EDE", "#E6E8FA", "#7D7F94", "#2E37FE", "#2F2F4F", "#42426F", "#8F8FBC", "#5959AB", "#7171C6", "#D9D9F3", "#23238E", "#3232CC", "#3232CD", "#191970", "#E6E6FA", "#000033", "#000080", "#00008B", "#00009C", "#0000CD", "#0000EE", "#0000FF", "#3333FF", "#4D4DFF", "#6666FF", "#AAAAFF"]

    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    date_range = pd.date_range(start_date, end_date)
    df_date_range = pd.DataFrame(index = date_range)
    # giant_table = giant_table[(start_date < giant_table[[feature_to_visualize]].index) 
    #             & (giant_table[[feature_to_visualize]].index < end_date)]

    for feature_to_visualize in features_to_visualize:
        print(feature_to_visualize)
        ax = ""
        fig, ax = plt.subplots(figsize = (int((end_date - start_date).days)/ 10 ,2))
        count1 = 0
        count2 = 0
        for pid, df_grp in table_groups:
            if (pid not in pids_to_visualize): continue
            df_grp = df_grp[(start_date < df_grp[[feature_to_visualize]].index) 
                        & (df_grp[[feature_to_visualize]].index < end_date)]
            df_grp = df_date_range.merge(df_grp, how = "left", left_index=True, right_index = True)
            if (flags_discrimination[pid]):
                color = colors1[count1 % len(colors1)]
                line1, = plt.plot(df_grp[[feature_to_visualize]].dropna(), linewidth = 0.5, color = color,
                    label = "label" + str(int(flags_discrimination[pid])) + str(count1))
                count1 += 1
                for date in dates_discrimination[pid]:
                    if (date < start_date or date > end_date): continue
                    y = df_grp[[feature_to_visualize]].loc[date]
                    plt.scatter(date,y, c = color, s = 15)

            else:
                color = colors2[count2 % len(colors2)]
                line2, = plt.plot(df_grp[[feature_to_visualize]].dropna(), linewidth = 0.5, color = color,
                    label = "label" + str(int(flags_discrimination[pid])) + str(count2))
                count2 += 1
            plt.title(feature_to_visualize)
        plt.xlim((start_date, end_date))
        plt.legend([line1, line2], ["Dis","No-Dis"])
        try:
            plt.xticks(rotation='vertical')
        except: pass
        figs.append(fig)
        if (save_fig_flag):
            if not os.path.exists(save_fig_path):
                os.mkdir(save_fig_path)
            plt.savefig(save_fig_path + "/" + feature_to_visualize + ".png", 
                        bbox_inches = "tight", dpi = 200)
        else:
            plt.show()
        plt.close()
        plt.cla()
        plt.clf()
    return figs 

In [ ]:
''' [Vis block]
The block to play with differnet parameters and number of pids to visualize
'''

# Random sample the pids to visualize
num_to_vis_in_each_group = 10

pids_to_vis = list(np.random.choice(dis_pids,num_to_vis_in_each_group, replace = False))+\
            list((np.random.choice(nodis_pids,num_to_vis_in_each_group, replace = False)))

# features to visualize
features_to_visualize = all_features_can_be_visualize[:]

# Window size to visualize
start_date = "1/25/2018"
end_date = "6/15/2018"

In [ ]:
for i,f in enumerate(all_features_can_be_visualize):
    if (f == "unfair_weight"):
        print(i)

In [ ]:
features_to_visualize = all_features_can_be_visualize[556:]

figs = plot_all_participants_together_per_feature(
    deepcopy(giant_table_groups),
    features_to_visualize,
    all_pids,
    flag_pids,
    dates_discrimination_pids,
    start_date,
    end_date,
    save_fig_flag = True,
    save_fig_path = path_to_save_figs + "/along_timeline"
)

# Visualize Discrimiation Events Before/After

In [ ]:
def plot_before_after_discrimination_per_feature(
    table_groups,
    features_to_visualize,
    pids_to_visualize,
    flags_discrimination,
    dates_discrimination,
    colors,
    date_range = 20,
    save_fig_flag = False,
    save_fig_path = path_to_save_figs + "/dis_events_beforeafter",
):
    '''
    Visualize all participants' together, one figure per feature
    
    :param DataFrame.groupby table_groups: big tables groupby on pids
    :param list features_to_visualize: a list of feature names (column names) to figures
    :param dict flags_discrimination: a dictionary including all pids about the flag, {key: pids, value: flags}
    :param dict dates_discrimination: a dictionary including all pids about the dates they experience about discrimination
                                            {key: pids who experienced discrimination, value: a list of dates}
    :param colorstring colors: a list of color string
    :param int date_range: window size of the visualization (number of days)
    :param bool save_fig_flag: true for save the figure, false to plot
    :param str save_fig_path: path to save figures, if plotted
    :return: a list of figures
    '''
    data_discrimination_events = {}
    for pid, dates in dates_discrimination.items():
        if (pid not in pids_to_visualize): continue
        table_pid = table_groups.get_group(pid)
        data_discrimination_events[pid] = []
        for date in dates:
            start = date - pd.to_timedelta(date_range / 2, unit='D')
            end = date + pd.to_timedelta(date_range / 2, unit='D')
            if (start < min_date): start = min_date
            if (end > max_date): end = max_date
            data_discrimination_events[pid].append(
                deepcopy(table_pid[
                    (start <= table_pid.index) & (table_pid.index <= end)
                ])
            )

    
    figs = []

    center_date = pd.to_datetime("2000/01/01")
    for feature_to_visualize in features_to_visualize:
        print(feature_to_visualize)
        fig, ax = plt.subplots(figsize = (date_range / 2,2))
        count = 0
        for pid, events in data_discrimination_events.items():
            color = colors[count % len(colors)]
            for event, date in zip(events, dates_discrimination[pid]):
                event_copy = deepcopy(event)
                date_distance = date - center_date
                event_copy.index = event_copy.index - date_distance
                ax.plot(event_copy[[feature_to_visualize]].dropna(), linewidth = 0.5, color = color)
            count += 1
        ax.set_title(feature_to_visualize)
        plt.axvline(x=center_date)
        plt.xticks(rotation='vertical')
        figs.append(fig)
        if (save_fig_flag):
            if not os.path.exists(save_fig_path):
                os.makedirs(save_fig_path)
            plt.savefig(save_fig_path + "/" + feature_to_visualize + ".png", 
                        bbox_inches = "tight", dpi = 200)
        else:
            plt.show()
        plt.cla()
        plt.clf()
        plt.close()

    return figs

In [ ]:
import random

colors = ["#CD2990", "#D02090", "#C71585", "#EE30A7", "#FF34B3", "#FF6EC7", "#F6A4D5", "#F7B3DA", "#543948", "#D4318C", "#8B0A50", "#CD1076", "#EE1289", "#FF1493", "#9D6B84", "#8B3A62", "#872657", "#DE85B1", "#551033", "#FF007F", "#FF69B4", "#FF6EB4", "#A46582", "#EE6AA7", "#B6316C", "#8B2252", "#CD3278", "#EE3A8C", "#FF3E96", "#CD6090", "#862A51", "#FF0066", "#8B8386", "#E0427F", "#FF92BB", "#A2627A", "#B03060", "#EEE0E5", "#CDC1C5", "#CD6889", "#DB7093", "#F6CCDA", "#FF82AB", "#FFF0F5", "#8B475D", "#EE799F", "#EAB5C5", "#A5435C", "#8B636C", "#CD919E", "#EEA9B8", "#FFB5C5", "#DC143C", "#F54D70", "#FF0033", "#F6A8B6", "#DB2645", "#FFC0CB", "#E79EA9", "#EEA2AD", "#99182C", "#E31230", "#FFADB9", "#FFB6C1", "#8B5F65", "#422C2F", "#CD8C95", "#DB9EA6", "#FFAEB9", "#CC4E5C", "#55141C", "#BB2A3C", "#C82536", "#AF1E2D", "#B81324", "#E32636", "#FA1D2F", "#F6C9CC", "#B0171F", "#EB5E66", "#D0A9AA", "#D41A1F", "#F64D54", "#FF030D", "#551011", "#E32E30", "#E33638", "#9E0508", "#E35152", 
               "#008B8B", "#00CDCD", "#00EEEE", "#00FFFF", "#00FFFF", "#97FFFF", "#BBFFFF", "#E0FFFF", "#F0FFFF", "#00CED1", "#5F9EA0", "#00868B", "#00C5CD", "#00E5EE", "#00F5FF", "#67E6EC", "#4A777A", "#05EDFF", "#53868B", "#73B1B7", "#05E9FF", "#7AC5CD", "#8EE5EE", "#05B8CC", "#98F5FF", "#B0E0E6", "#C1F0F6", "#39B7CD", "#65909A", "#0EBFE9", "#C3E4ED", "#68838B", "#63D1F4", "#9AC0CD", "#50A6C2", "#ADD8E6", "#B2DFEE", "#00688B", "#009ACD", "#0099CC", "#00B2EE", "#00BFFF", "#BFEFFF", "#33A1C9", "#507786", "#87CEEB", "#38B0DE", "#0BB5FF", "#42C0FB", "#6996AD", "#539DC2", "#236B8E", "#3299CC", "#0198E1", "#33A1DE", "#607B8B", "#35586C", "#5D92B1", "#8DB6CD", "#325C74", "#A4D3EE", "#82CFFD", "#67C8FF", "#B0E2FF", "#87CEFA", "#6CA6CD", "#4A708B", "#9BC4E2", "#7EC0EE", "#87CEFF", "#517693", "#5D7B93", "#42647F", "#4682B4", "#4F94CD", "#5CACEE", "#63B8FF", "#525C65", "#36648B", "#62B1F6", "#74BBFB", "#F0F8FF", "#4E78A0", "#0D4F8B", "#708090", "#708090", "#778899", "#778899", "#6183A6", "#9FB6CD", "#7D9EC0", "#104E8B", "#1874CD", "#1C86EE", "#60AFFE", "#007FFF", "#1E90FF", "#6C7B8B", "#B7C3D0", "#739AC5", "#75A1D0", "#B9D3EE", "#499DF5", "#C6E2FF", "#3B6AA0", "#7AA9DD", "#0276FD", "#003F87", "#6E7B8B", "#506987", "#A2B5CD", "#4372AA", "#26466D", "#1D7CF2", "#687C97", "#344152", "#50729F", "#4973AB", "#B0C4DE", "#3063A5", "#BCD2EE", "#7EB6FF", "#CAE1FF", "#4D71A3", "#2B4F81", "#4981CE", "#88ACE0", "#5993E5", "#3A66A7", "#3579DC", "#5190ED", "#42526C", "#4D6FAC", "#2E5090", "#2C5197", "#6495ED", "#6D9BF1", "#5B90F6", "#1464F4", "#3A5894", "#7093DB", "#1B3F8B", "#5971AD", "#0147FA", "#3D59AB", "#27408B", "#3A5FCD", "#4169E1", "#436EEE", "#003EFF", "#4876FF", "#A9ACB6", "#22316C", "#162252", "#3B4990", "#283A90", "#6F7285", "#838EDE", "#E6E8FA", "#7D7F94", "#2E37FE", "#2F2F4F", "#42426F", "#8F8FBC", "#5959AB", "#7171C6", "#D9D9F3", "#23238E", "#3232CC", "#3232CD", "#191970", "#E6E6FA", "#000033", "#000080", "#00008B", "#00009C", "#0000CD", "#0000EE", "#0000FF", "#3333FF", "#4D4DFF", "#6666FF", "#AAAAFF"]
random.shuffle(colors)

In [ ]:
''' [Vis block]
The block to play with differnet parameters and number of pids to visualize
'''

#Random sample the pids to visualize
num_to_vis = 10
pids_to_vis = list(np.random.choice(dis_pids,num_to_vis, replace = False))

# Feature to be visualized
features_to_visualize = all_features_can_be_visualize

# Number of days to show the dynamics
date_range = 20

pids_to_vis = list(np.random.choice(dis_pids,num_to_vis, replace = False))
plot_before_after_discrimination_per_feature(
    deepcopy(giant_table_groups),
    features_to_visualize,
    dis_pids,
    flag_pids,
    dates_discrimination_pids,
    colors,
    date_range = 20,
    save_fig_flag = True,
    save_fig_path = path_to_save_figs + "/dis_events_beforeafter",
)

In [ ]:
def plot_single_user_with_nan(data, ax, color):
    data_mk = np.ma.masked_where(np.isnan(data),data)
    overall_len = len(data_mk)
    def find_continuous_nans(start_index, mask_list):
        if (start_index == 0 and not mask_list[start_index]):
            return 0
        count = 1
        assert(mask_list[start_index])
        while((start_index + count) < overall_len and mask_list[start_index + count]):
            count += 1
        return count
    index = find_continuous_nans(0, data_mk.mask)
    while (index < (overall_len - 1)):
        m_next = data_mk.mask[index + 1]
        if (m_next): #if self is nan
            na_len = find_continuous_nans(index + 1, data_mk.mask)
            if ((index + na_len + 1) < overall_len):
                assert(not data_mk.mask[index + na_len + 1])
                ax.plot([index, index+na_len+1], [data_mk.data[index], data_mk.data[index+na_len+1]],"--", color = color, linewidth = 0.5)
        else:
            na_len = 0
            ax.plot([index, index+na_len+1], [data_mk.data[index], data_mk.data[index+na_len+1]],"-", color = color, linewidth = 0.5)
        index += na_len + 1
    return ax


def plot_users_with_nan(dfgrps, table_name, feature_cols):
    figs = []
    for feature_col in feature_cols:
        count = 0
        colors1 = ["#CD2990", "#D02090", "#C71585", "#EE30A7", "#FF34B3", "#FF6EC7", "#F6A4D5", "#F7B3DA", "#543948", "#D4318C", "#8B0A50", "#CD1076", "#EE1289", "#FF1493", "#9D6B84", "#8B3A62", "#872657", "#DE85B1", "#551033", "#FF007F", "#FF69B4", "#FF6EB4", "#A46582", "#EE6AA7", "#B6316C", "#8B2252", "#CD3278", "#EE3A8C", "#FF3E96", "#CD6090", "#862A51", "#FF0066", "#8B8386", "#E0427F", "#FF92BB", "#A2627A", "#B03060", "#EEE0E5", "#CDC1C5", "#CD6889", "#DB7093", "#F6CCDA", "#FF82AB", "#FFF0F5", "#8B475D", "#EE799F", "#EAB5C5", "#A5435C", "#8B636C", "#CD919E", "#EEA9B8", "#FFB5C5", "#DC143C", "#F54D70", "#FF0033", "#F6A8B6", "#DB2645", "#FFC0CB", "#E79EA9", "#EEA2AD", "#99182C", "#E31230", "#FFADB9", "#FFB6C1", "#8B5F65", "#422C2F", "#CD8C95", "#DB9EA6", "#FFAEB9", "#CC4E5C", "#55141C", "#BB2A3C", "#C82536", "#AF1E2D", "#B81324", "#E32636", "#FA1D2F", "#F6C9CC", "#B0171F", "#EB5E66", "#D0A9AA", "#D41A1F", "#F64D54", "#FF030D", "#551011", "#E32E30", "#E33638", "#9E0508", "#E35152"]
        colors2 = ["#008B8B", "#00CDCD", "#00EEEE", "#00FFFF", "#00FFFF", "#97FFFF", "#BBFFFF", "#E0FFFF", "#F0FFFF", "#00CED1", "#5F9EA0", "#00868B", "#00C5CD", "#00E5EE", "#00F5FF", "#67E6EC", "#4A777A", "#05EDFF", "#53868B", "#73B1B7", "#05E9FF", "#7AC5CD", "#8EE5EE", "#05B8CC", "#98F5FF", "#B0E0E6", "#C1F0F6", "#39B7CD", "#65909A", "#0EBFE9", "#C3E4ED", "#68838B", "#63D1F4", "#9AC0CD", "#50A6C2", "#ADD8E6", "#B2DFEE", "#00688B", "#009ACD", "#0099CC", "#00B2EE", "#00BFFF", "#BFEFFF", "#33A1C9", "#507786", "#87CEEB", "#38B0DE", "#0BB5FF", "#42C0FB", "#6996AD", "#539DC2", "#236B8E", "#3299CC", "#0198E1", "#33A1DE", "#607B8B", "#35586C", "#5D92B1", "#8DB6CD", "#325C74", "#A4D3EE", "#82CFFD", "#67C8FF", "#B0E2FF", "#87CEFA", "#6CA6CD", "#4A708B", "#9BC4E2", "#7EC0EE", "#87CEFF", "#517693", "#5D7B93", "#42647F", "#4682B4", "#4F94CD", "#5CACEE", "#63B8FF", "#525C65", "#36648B", "#62B1F6", "#74BBFB", "#F0F8FF", "#4E78A0", "#0D4F8B", "#708090", "#708090", "#778899", "#778899", "#6183A6", "#9FB6CD", "#7D9EC0", "#104E8B", "#1874CD", "#1C86EE", "#60AFFE", "#007FFF", "#1E90FF", "#6C7B8B", "#B7C3D0", "#739AC5", "#75A1D0", "#B9D3EE", "#499DF5", "#C6E2FF", "#3B6AA0", "#7AA9DD", "#0276FD", "#003F87", "#6E7B8B", "#506987", "#A2B5CD", "#4372AA", "#26466D", "#1D7CF2", "#687C97", "#344152", "#50729F", "#4973AB", "#B0C4DE", "#3063A5", "#BCD2EE", "#7EB6FF", "#CAE1FF", "#4D71A3", "#2B4F81", "#4981CE", "#88ACE0", "#5993E5", "#3A66A7", "#3579DC", "#5190ED", "#42526C", "#4D6FAC", "#2E5090", "#2C5197", "#6495ED", "#6D9BF1", "#5B90F6", "#1464F4", "#3A5894", "#7093DB", "#1B3F8B", "#5971AD", "#0147FA", "#3D59AB", "#27408B", "#3A5FCD", "#4169E1", "#436EEE", "#003EFF", "#4876FF", "#A9ACB6", "#22316C", "#162252", "#3B4990", "#283A90", "#6F7285", "#838EDE", "#E6E8FA", "#7D7F94", "#2E37FE", "#2F2F4F", "#42426F", "#8F8FBC", "#5959AB", "#7171C6", "#D9D9F3", "#23238E", "#3232CC", "#3232CD", "#191970", "#E6E6FA", "#000033", "#000080", "#00008B", "#00009C", "#0000CD", "#0000EE", "#0000FF", "#3333FF", "#4D4DFF", "#6666FF", "#AAAAFF"]
#         colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#000000']
        fig, ax = plt.subplots()
        for key, group in data_dfgrps["f_batt"]:
            print(key)
            data = group[feature_col]
            data = filtering(data, 0, 1440)
            if (postDep(group)):
                colors = colors1
            else:
                colors = colors2
            plot_single_user_with_nan(data, ax, colors[count%len(colors)])
            if (count > 2):
                break
            count += 1
        figs.append(fig)
    return figs